# 1. Tiedonkeruu ja esikäsittely

## 1.1 Rajapinnat

VOOKA-projektin ETL-työkalussa tietojen kerääminen on mahdollista teknisesti kolmesta eri lähteestä:

1. OGC:n WFS-standardin mukaisista rajapinnoista,
2. Esrin ArcGIS Feature Layereista sekä
3. OGC API Features -rajapinnoista

Kaksi ensimmäiseksi mainittua palauttavat syötetyn URL:n mukaisen tiedon kokonaisuudessaan GeoPandas GeoDataframena.

### 1.1.1 WFS ja ArcGIS Feature Layer

In [ ]:
# Käytettävien Python moduulien ja kehitettyjen funktioiden sisäänluku
import geopandas as gpd
from lib.get_feature_data import getWFSlayers, getWFSdata, getArcgisFeatureLayer

In [ ]:
# Esimerkki 1.
print(getWFSlayers(url="https://kunnat.navici.com/geoserver/sulkava/wfs"))
yk_data = getWFSdata(url="https://kunnat.navici.com/geoserver/sulkava/wfs", layer="sulkava:kaavarajat_osayleiskaava")
print(type(yk_data))

# Tallennus
yk_data.to_file(r"<insert filepath here>.gpkg", layer="<insert layer name here>")

In [ ]:
# Esimerkki 2.
rak_data = getArcgisFeatureLayer(layer_url="https://services1.arcgis.com/biwBTnEqa75tyWMg/ArcGIS/rest/services/RAK_indeksi/FeatureServer/0")
print(type(rak_data))

# Tallennus
rak_data.to_file(r"<insert filepath here>.gpkg", layer="<insert layer name here>")

### 1.1.2 OGC API Features

OGC API Features -skripti palauttaa API:n sisältöä GeoJSON-formaatissa Etelä-Savon maankunnan rajauksella (bounding box), joka on kovakoodattu toteutukseen. **Jatkokehityksessä kovakoodattu maantieteellinen rajaus tulee korvata halutun alueen koordinaateilla!**

Skiptiin on myös lisätty apufunktioita, joiden avulla MML:n kiinteistötietorajapinnasta haettu GeoJSON-muotoinen dataskeema voidaan normalisoida tabulaariseksi tiedoksi GeoPandas GeoDataframeen ja tallentaa pickle-kirjaston pkl-tiedostona.

In [ ]:
# Kehitettyjen funktioiden sisäänluku
from lib.OGC_API_Features_handler import getDataFromAPI, normalizeKiinteistorajaSchema, normalizeKiinteistotunnusSchema, normalizeRajamerkkiSchema

In [ ]:
# Esimerkki 3.
# Kirjoita parametreihin halutut tiedot (lisätietoja OGC_API_Features_handler.py funktioiden docstringeista)
# Tarvitset kiinteistötunnustietoja (piste) sekä kiinteistöpalstatietoja (polygon) myöhemmin!
api_data = getDataFromAPI(url="<insert url here>",
                          username="<insert username here>",
                          password="<insert password here>",
                          outfp=r"<insert output filepath here>.pkl")

# MML:n kiinteistötietorajapinnan tietojen normalisointi ja pkl-tallentaminen
norm_data_1 = normalizeKiinteistotunnusSchema(infp=r"<insert input filepath here>.pkl", outfp=r"<insert output filepath here>.pkl")
#norm_data_2 = normalizeKiinteistorajaSchema(infp="<insert input filepath here>", outfp="<insert output filepath here>")
#norm_data_3 = normalizeRajamerkkiSchema(infp="<insert input filepath here>", outfp="<insert output filepath here>")

## 1.2 Esikäsittely

### 1.2.1 KTJ-aineisto

MML:n KTJ-aineistoissa kuntaliitosalueiden kaavoille on ilmoitettu vanha kuntakoodi. Nämä tulee päivittää vastaamaan voimassa olevaa kuntakoodia kattavan vertailun mahdollistamiseksi kunta-aineistojen kanssa. **Toteutus on koodattu vastaamaan ainoastaan Etelä-Savon maakuntien kuntakoodeja!** Input parametrina käytetään GeoPandas GeoDataframe muodossa olevaa KTJ-aineistoa.

In [ ]:
# Kehitettyjen funktioiden sisäänluku
from lib.ktj_update_kuntakoodit import updateOldKuntakoodi

In [ ]:
# Lisää tiedostopolut KTJ-aineiston input ja output geopackageen sekä karttataso
ktj_data = gpd.read_file(r"<insert filepath here>.gpkg", layer="<insert layer name here>")
ktj_update = updateOldKuntakoodi(ktj_data)

# Tallennus
ktj_update.to_file(r"<insert filepath here>.gpkg", layer="<insert layer name here>")

### 1.2.2 Kunta-aineistot

Kunta-aineistojen esikäsittelytarpeet voivat olla hyvin kuntaspesifejä. VOOKA-projektin tapauksessa erään kunnan WFS-rajapinta oli toteutettu KuntaGML-formaatissa, jonka sisäänluku ei onnistunut perinteisin menetelmin. Ongelma ratkaistiin erillisellä XML-parser-skriptillä, jossa kaavatiedot irrotettiin suoraan merkintäkielen rakenteesta.

Lisäksi osa kunta-aineistoista voi olla saatavina vain erillistoimitettuina CAD-piirroksina. VOOKA-projetissa erään kunnan CAD-aineistossa ominaisuustiedot oli sidottu pistegeometrioihin varsinaisten kaavarajojen sijasta. Ominaisuustietojen yhdistäminen kaavarajoihin toteutettiin hankkeessa niin ikään erillisellä skriptillä, jota edelsi tiedon muunnos paikkatietoformaattiin FME-ohjelmistolla.

Näihin tapauksiin voi käydä tutustumassa erikseen VOOKA-hankkeen [GitHub-sivujen kautta](https://github.com/ubigu/vooka). Erillisskriptit löytyvät muut-kansiossa.